In [11]:
import geopandas as gpd
import pandas as pd
import requests
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource
from bokeh.plotting import figure, output_file
from bokeh.io import output_notebook, show
from IPython.display import display
from pandas import json_normalize
import folium
import json
from pyproj import CRS
import matplotlib.pyplot as plt 
import contextily as ctx
import numpy as np
import plotly.express as px
from matplotlib.cm import ScalarMappable
from folium import plugins
import dash
import dash_core_components as dcc
import dash_html_components as html
from jupyter_dash import JupyterDash
import dash
from dash.dependencies import Input, Output, State
from geopy.geocoders import Nominatim
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider
from bokeh.plotting import figure
from bokeh.themes import Theme
from bokeh.io import show, output_notebook
from bokeh.tile_providers import get_provider
from bokeh.models import ColumnDataSource, HoverTool, Select, CustomJS
import plotly.graph_objects as go
import plotly.express as px
output_notebook()


#-------------------------datas-------------------------
response= requests.get("http://127.0.0.1:5000/measurements")
rawdata=response.text
data=json.loads(rawdata)
dff=pd.json_normalize(data)
dff
df_measurements = dff[dff['value'] != -9999]
df_measurements = df_measurements[df_measurements['value'] < 1000]
df_measurements.sort_values(by='value')

response1=requests.get("http://127.0.0.1:5000/sensor")
rawdata1=response1.text
rawdata1
df1=json.loads(rawdata1)
df1=pd.json_normalize(df1)
#df1 = df1.drop('geom', axis=1)
df1


gdf1_sensor = gpd.GeoDataFrame(df1, geometry=gpd.points_from_xy(df1.lng, df1.lat))
output_path = 'F:\polimi\software engineering\prj output mol/Stations.gpkg'
gdf1_sensor.to_file(output_path, driver='GPKG')
target_crs = CRS.from_epsg(4326) 
gdf1_sensor.crs = target_crs
print(gdf1_sensor.crs)
gdf1_sensor


gdf_sen_meas = gdf1_sensor.merge(df_measurements, on='sensorid', how='inner')

import datetime

has_nan = gdf_sen_meas.isna().any().any()
if has_nan:
    print("The DataFrame contains NaN values.")
else:
    print("The DataFrame does not contain any NaN values.")
gdf_sen_meas=gdf_sen_meas.dropna()


gdf_sen_meas['date'] = pd.to_datetime(gdf_sen_meas['date'], format='%d/%m/%Y %I:%M:%S %p')
unique_weeks = sorted(gdf_sen_meas['date'].dt.week.unique().tolist())
gdfdate= gdf_sen_meas['date'].dt.date
gdf_sen_meas.sort_values(by='date')
#lombardy=gpd.read_file('/Users/moldirseytqazy/Downloads/software_geo-main/resources/Lombardy.gpkg')


town_names = gdf_sen_meas['town'].unique().tolist()



logged_user_id = None

def get_user(username):
    response = requests.get("http://127.0.0.1:5000/user")
    user_data = response.json()
    for user in user_data:
        if user["name"] == username:
            return user
    return None

def login_user(username):
    global logged_user_id
    user = get_user(username)
    if user is None:
        return html.Span("Not correct. Try again or create a new user", style={ 'margin-left': '80px', 'margin-top': '10px', 'color': 'darkgray'})
    else:
        logged_user_id = user["id"]
        return html.Span("Success" , style={ 'margin-left': '80px', 'margin-top': '10px','color': 'darkgray'})

def get_logged_user_id():
    return logged_user_id

def register_user(username):
    user = get_user(username)
    if user is None:
        user_json = {"name": username}
        headers = {"Content-Type": "application/json"}
        res = requests.post("http://127.0.0.1:5000/user", json=user_json, headers=headers)
        if res.status_code == 201:
            return html.Span("Success. Welcome {}".format(username), style={ 'margin-left': '80px', 'margin-top': '10px','color': 'darkgray'})
        else:
            return html.Span("Try again" , style={ 'margin-left': '80px', 'margin-top': '10px','color': 'darkgray'})
    else:
        return html.Span("Your username already exists, try logging in.", style={ 'margin-left': '80px', 'margin-top': '10px','color': 'darkgray'})
    
api_url = 'http://127.0.0.1:5000/favorites'
#user_id = None
nearest_station_result = None
geolocator = Nominatim(timeout=10, user_agent="my-app")


    
    




app = JupyterDash(__name__)
#----------------------------------------------------------------------------------------------
# Create the animation frames



# Create the animation figure
figan = px.scatter(gdf_sen_meas, x="lng", y="lat", animation_frame=gdfdate, color="value",
                 range_x=[gdf_sen_meas['lng'].min(), gdf_sen_meas['lng'].max()],
                 range_y=[gdf_sen_meas['lat'].min(), gdf_sen_meas['lat'].max()],
                 title="Air Quality Animation")

figan.update_layout(xaxis_title="Longitude", yaxis_title="Latitude")

#---------------------------------------------------------------------------------------
gdfdate= pd.to_datetime(gdfdate)
gdf_sen_meas = gdf_sen_meas.set_index(gdfdate)
gdf_sen_meas.index = pd.DatetimeIndex(gdf_sen_meas.index)
gdf_weekly = gdf_sen_meas.resample('W').mean()
gdf_weekly = gdf_weekly.reset_index()
#----------------------------------------------------------------------------

scatter = go.Scatter(
    x=gdf_weekly['date'],
    y=gdf_weekly['value'],
    mode='markers',
    marker=dict(
        color='red',
        size=8,
        symbol='circle'
    )
)

layout = go.Layout(
    title='Date vs. Value',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Value'),
)

fig = go.Figure(data=[scatter], layout=layout)


     


    
def another_function():
    user_id = get_logged_user_id()
    if user_id is not None:
        #print(user_id)
        return int(user_id)# Perform other operations with the logged user ID
    else:
        print("No user logged inn")


def get_filtered_data():
    global user_id

    if logged_user_id is not None:
        user_id = logged_user_id
    else:
        print("User is not logged in.")
        return []

    response = requests.get(api_url)

    if response.status_code == 200:
        data = response.json()
        filtered_data = list({item['sensorid']: item for item in data if item['userid'] == user_id}.keys())
        return filtered_data
    else:
        print("Failed to retrieve data from the API.")
        return []
    




#--------------------------------layout -----------------------------------
app.layout = html.Div(
    style={
        'backgroundColor': '#FFFFFF',
        'display': 'flex',
        'flexDirection': 'column',
        'alignItems': 'center',
        'height': '100vh'
    },
    children=[
        html.H1(
            'AIR QUALITY DASHBOARD',
            style={
                'textAlign': 'center',
                'marginTop': '0',
                'backgroundColor': '#FFA500',
                'border': '2px solid black',
                'padding': '10px',
                'color': 'black',
                'font-size': '24px',
                'font-weight': 'bold'
            }
        ),
        html.Div(
            style={
                'margin-top': '0px',
                'display': 'flex',
                'width': '100%',
                'font-family': 'Calibre, Arial, sans-serif'
            },
            children=[
                html.Div(
                    style={'margin': '0px'},
                    children=[
                        html.Label(
                            "Username:",
                            style={
                                'color': '#525252',
                                'font-size': '16px',
                                'margin-right': '10px'
                            }
                        ),
                        dcc.Input(
                            id="username-input",
                            type="text",
                            style={
                                'padding': '8px',
                                'border-radius': '4px',
                                'background-color': 'white',
                                'border': '1px solid #b7b7b7',
                                'color': '#525252',
                                'font-size': '16px'
                            }
                        ),
                        html.Button(
                            "Login",
                            id="login-button",
                            n_clicks=0,
                            style={
                                'font-size': '16px',
                                'padding': '8px',
                                'border-radius': '4px',
                                'background-color': '#e8e8e8',
                                'border': '1px solid #b7b7b7',
                                'color': '#8c8c8c',
                                'cursor': 'pointer',
                                'transition': 'background-color 0.3s ease',
                                'margin-top': '0px',
                                'margin-bottom': '10px',
                                'margin-left': '10px',
                                'margin-right': '5px'
                            }
                        ),
                        html.Button(
                            "Register",
                            id="register-button",
                            n_clicks=0,
                            style={
                                'font-size': '16px',
                                'padding': '8px',
                                'border-radius': '4px',
                                'background-color': '#e8e8e8',
                                'border': '1px solid #b7b7b7',
                                'color': '#8c8c8c',
                                'cursor': 'pointer',
                                'transition': 'background-color 0.3s ease',
                                'margin-top': '0px',
                                'margin-bottom': '10px',
                                'margin-left': '10px',
                                'margin-right': '0px'
                            }
                        ),
                        html.Div(
                            id="status-label",
                            style={
                                'font-size': '16px',
                                'margin-left': '10px',
                                'color': 'darkgray'
                            }
                        )
                    ]
                )
            ]
        ),
        html.Div(
            style={
                'display': 'flex',
                'width': '100%',
                'font-family': 'Calibre, Arial, sans-serif'
            },
            children=[
                html.Div(
                    style={'margin': '0px'},
                    children=[
                        html.H1(""),
                        dcc.Input(
                            id="address-input",
                            type="text",
                            placeholder="Enter address",
                            style={
                                'margin-left': '10px',
                                'padding': '8px',
                                'border-radius': '4px',
                                'background-color': 'white',
                                'border': '1px solid #b7b7b7',
                                'color': '#525252',
                                'font-size': '16px'
                            }
                        ),
                        html.Button(
                            id="find-button",
                            n_clicks=0,
                            children="Find nearest station",
                            style={
                                'font-size': '16px',
                                'padding': '8px',
                                'border-radius': '4px',
                                'background-color': '#e8e8e8',
                                'border': '1px solid #b7b7b7',
                                'color': '#8c8c8c',
                                'cursor': 'pointer',
                                'transition': 'background-color 0.3s ease',
                                'margin-top': '0px',
                                'margin-bottom': '10px',
                                'margin-left': '10px',
                                'margin-right': '10px'
                            }
                        ),
                        html.Button(
                            id="add-favorites-button",
                            n_clicks=0,
                            children="Add favorites",
                            style={
                                'font-size': '16px',
                                'padding': '8px',
                                'border-radius': '4px',
                                'background-color': '#e8e8e8',
                                'border': '1px solid #b7b7b7',
                                'color': '#8c8c8c',
                                'cursor': 'pointer',
                                'transition': 'background-color 0.3s ease',
                                'margin-top': '0px',
                                'margin-bottom': '10px',
                                'margin-right': '10px'
                            }
                        ),
                        html.Button(
                            'Show Favorites',
                            id='show-button',
                            n_clicks=0,
                            style={
                                'font-size': '16px',
                                'padding': '8px',
                                'border-radius': '4px',
                                'background-color': '#e8e8e8',
                                'border': '1px solid #b7b7b7',
                                'color': '#8c8c8c',
                                'cursor': 'pointer',
                                'transition': 'background-color 0.3s ease',
                                'margin-top': '10px',
                                'margin-bottom': '0px',
                                'margin-left': '10px',
                                'margin-right': '10px'
                            }
                        )
                    ]
                )
            ]
        ),
        html.Div(
            style={'display': 'flex', 'width': '100%'},
            children=[
                html.Div(
                    style={'margin': '10px', 'flex': '10%'},
                    children=[
                        html.Div(id='item-list')
                    ]
                ),
                html.Div(
                    style={'flex': '100%', 'margin': '10px'},
                    children=[
                        html.Div(id="output-container")
                    ]
                ),
            ]
        ),
        html.Div(
            style={'display': 'flex', 'width': '100%'},
            children=[
                html.Div(
                    style={'flex': '1'},
                    children=[
                        dcc.Dropdown(
                            id='town-search',
                            options=[{'label': town, 'value': town} for town in town_names],
                            placeholder='Search town',
                            style={'width': '300px', 'backgroundColor': 'white'}
                        ),
                        html.Div(id='map-container')
                    ]
                ),
                html.Div(
                    style={'flex': '1', 'marginLeft': '20px'},
                    children=[
                        dcc.Dropdown(
                            id='week-search',
                            options=[{'label': week, 'value': week} for week in unique_weeks],
                            placeholder='Search week',
                            style={'width': '300px', 'backgroundColor': 'white'}
                        ),
                        html.Div(id='value-container', style={'width': '630px'})
                    ]
                )
            ]
        ),
        html.Div(
            style={'display': 'flex', 'width': '100%'},
            children=[
                html.Div(
                    style={'flex': '1'},
                    children=[
                        html.H2('Graph 1'),
                        dcc.Graph(figure=figan)
                    ]
                ),
                html.Div(
                    style={'flex': '1', 'marginLeft': '20px'},
                    children=[
                        html.H2('Graph 2'),
                        dcc.Graph(figure=fig)
                    ]
                )
            ]
        )
    ]
)





                    
        
    
def get_geolocation(address):
    location = geolocator.geocode(address)
    if location:
        latitude = location.latitude
        longitude = location.longitude
        return latitude, longitude
    else:
        return None

def get_api_response(latitude, longitude):
    # Replace 'YOUR_API_ENDPOINT' with the actual endpoint of your API
    api_endpoint = 'http://127.0.0.1:5000/sensor/findclose'

    # Create a dictionary with the latitude and longitude values
    payload = {"lat": longitude, "lng": latitude, "geom": True}

    try:
        # Make a GET request to the API endpoint with the JSON payload
        response = requests.get(api_endpoint, json=payload)

        # Check the response status code to ensure the request was successful
        if response.status_code == 200:
            # Process the response data as needed
            api_response = response.json()
            # Return the API response
            return api_response
        else:
            # Handle any errors that occurred during the API request
            print(f"Error: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        # Handle exceptions related to the request
        print(f"Error: {e}")
        return None  
  
    
    
    
                       
                
            
           
        
    


# Define the callback function for updating the map
@app.callback(
    Output('map-container', 'children'),
    [Input('town-search', 'value')]
)
def update_map(selected_town):
    if selected_town:
        town_data = gdf_sen_meas[gdf_sen_meas['town'] == selected_town]
        lat = town_data['lng'].mean()
        lon = town_data['lat'].mean()
        #geometry = town_data['geometry'].iloc[0]
        value = town_data['value'].mean()

        
        map = folium.Map(location=[lat, lon], zoom_start=12)

        marker = folium.Marker(location=[lat, lon], popup=f"{selected_town}: {value}")
        marker.add_to(map)

        legend_html = f"""
            <div style="position: fixed; 
                        upper: 50px; left: 50px; width: 150px; height: 80px; 
                        border:2px solid grey; z-index:9999; font-size:14px;
                        background-color:white; opacity: 0.9;
                        ">
                <b>Selected town:</b><br>
                {selected_town}<br>
                <b>Value:</b> {value}
            </div>
        """
        map.get_root().html.add_child(folium.Element(legend_html))

        return html.Iframe(srcDoc=map._repr_html_(), width='100%', height='300px')
    else:
        return ''

# -------------------------------callback function -----------------------------
@app.callback(
    Output('value-container', 'children'),
    [Input('town-search', 'value'),
     Input('week-search', 'value')]
)
def update_value(selected_town, selected_week):
    if selected_town and selected_week:
        town_data = gdf_sen_meas[(gdf_sen_meas['town'] == selected_town) & (gdf_sen_meas['date'].dt.week == selected_week)]
        value = town_data['value'].mean()

        # Make the request to the API with value
        response = requests.get(f"http://127.0.0.1:5000/status/{value}")
        message = response.text

        return html.Div([
            html.H3("Air quality value in the selected week:",style={'border': '1px solid black','backgroundColor': 'white'}),
            html.P(value,style={'border': '1px solid black','backgroundColor': 'white'}),
            html.H3("A masseege for you :",style={'border': '1px solid black','backgroundColor': 'white',}),
            html.P(message , style={'border': '1px solid black','backgroundColor': 'white'})
        ])
    else:
        return ''
# -------------------------------callback function -----------------------------
@app.callback(
    Output("status-label", "children"),
    [Input("login-button", "n_clicks"), Input("register-button", "n_clicks")],
    [State("username-input", "value")]
)
def update_status_label(login_clicks, register_clicks, username):
    changed_id = [p["prop_id"] for p in dash.callback_context.triggered][0]
    if "login-button" in changed_id:
        return login_user(username)
    elif "register-button" in changed_id:
        return register_user(username)
    else:
        return ""
@app.callback(
    dash.dependencies.Output("output-container", "children"),
    [dash.dependencies.Input("find-button", "n_clicks")],
    [dash.dependencies.State("address-input", "value")],
)
def find_nearest_station(n_clicks, address):
    global nearest_station_result  # Use the global variable
    if address:
        result = get_geolocation(address)
        if result:
            latitude, longitude = result
            api_response = get_api_response(latitude, longitude)
            if api_response:
                properties = api_response.get("properties", {})
                sensor_id = properties.get("sensorid")
                station_name = properties.get("stationname")
                province = properties.get("province")
                town = properties.get("town")
                map_center = [latitude, longitude]
                map_zoom = 12
                folium_map = folium.Map(location=map_center, zoom_start=map_zoom)

                for feature in api_response["features"]:
                    properties = feature["properties"]
                    coordinates = feature["geometry"]["coordinates"]
                    marker_geojson = {
                        "type": "Feature",
                        "geometry": {
                            "type": "Point",
                            "coordinates": coordinates,
                        },
                        "properties": {
                            "Title": properties["stationname"],
                            "Description": f"Sensor ID: {properties['sensorid']}\nProvince: {properties['province']}\nTown: {properties['town']}",
                        },
                    }
                    folium.GeoJson(
                        json.dumps(marker_geojson),
                        name="marker",
                        style_function=lambda x: {
                            "color": "red",
                            "weight": 2,
                            "fillOpacity": 0.8,
                        },
                        tooltip=folium.GeoJsonTooltip(
                            fields=["Title", "Description"]
                        ),
                    ).add_to(folium_map)

                marker_bounds = folium_map.get_bounds()
                folium_map.fit_bounds(marker_bounds)
                nearest_station_result = api_response  # Store the result in the global variable

                return html.Div(
                    [
                        html.H3(""),
                        html.P(
                            ""
                        ),
                        html.Iframe(
                            srcDoc=folium_map._repr_html_(),
                            style={'width': '95%', 'height': '450px', 'border': 'none', 'box-shadow': '0px 2px 8px rgba(0, 0, 0, 0.1)'},
                        ),
                    ]
                )
            else:
                return html.Div("Error occurred during API request.")
        else:
            return html.Div("Address not found.")
    else:
        return html.Div(".")


# Define the callback for adding favorites
@app.callback(
    dash.dependencies.Output("add-favorites-button", "n_clicks"),
    [dash.dependencies.Input("add-favorites-button", "n_clicks")],
)
def add_favorites(n_clicks):
    global nearest_station_result  # Use the global variable
    if nearest_station_result:
        closest_sensor_id = (
            nearest_station_result["features"][0]["properties"]["sensorid"]
        )
        user_id = get_logged_user_id()
        post_data = {"userid": user_id, "sensorid": closest_sensor_id}
        url = "http://127.0.0.1:5000/favorites"
        response = requests.post(url=url, json=post_data)
        response_status = response.status_code
        #print("Add favorites")
        return n_clicks
    else:
        return 0



    
@app.callback(
    Output('item-list', 'children'),
    [Input('show-button', 'n_clicks')]
)
def update_item_list(n_clicks):
    if n_clicks is not None:
        filtered_data = get_filtered_data()
        items = [html.P("Sensorid: " + str(sensor_id)) for sensor_id in filtered_data]
        return items

    return []

    
# Run the app
if __name__ == '__main__':
    app.run_server(mode='inline', port=8054)

Loading BokehJS ...

EPSG:4326
The DataFrame does not contain any NaN values.
Dash is running on http://127.0.0.1:8054/



User is not logged in.
